# Interactive Stock Screener System

This notebook implements an engaging and interactive stock screening system based on concepts from Chapter 4 of [Practical Python for Effective Algorithmic Trading](https://www.amazon.com/dp/B0F3S8FQ7C).

The screener analyzes stocks based on both technical indicators (price vs. moving averages) and fundamental metrics (P/E ratio, debt-to-equity ratio, dividend yield) to categorize them into four rating groups:
- Strong Buy 🔥
- Moderate Buy ✅
- Hold ⏹️
- Sell ❌

## Features

- Color-coded output for easier interpretation
- Visual scoring system with bar charts
- Detailed technical and fundamental analysis
- Interactive filtering options
- Comprehensive summary statistics

In [5]:
# Import required libraries (only using standard library)
import time
from IPython.display import clear_output, display, HTML
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
# Helper functions for visualization and display

def display_title():
    """Display a colorful title for the stock screener"""
    display(HTML("""
    <div style="background-color:#004466; color:#ffffff; padding:20px; border-radius:10px; text-align:center;">
        <h1 style="margin:0;">📊 Interactive Stock Screener 📊</h1>
        <p style="margin:0;">Based on Chapter 4 of 'Practical Python for Effective Algorithmic Trading'</p>
    </div>
    """))

def display_section(title):
    """Display a section header"""
    display(HTML(f"""
    <div style="background-color:#6699cc; color:white; padding:10px; border-radius:5px; margin-top:20px; margin-bottom:10px;">
        <h2 style="margin:0;">{title}</h2>
    </div>
    """))

def color_rating(rating):
    """Return HTML with colored rating"""
    if "Strong Buy" in rating:
        return f'<span style="color:green; font-weight:bold;">{rating} 🔥</span>'
    elif "Moderate Buy" in rating:
        return f'<span style="color:blue; font-weight:bold;">{rating} ✅</span>'
    elif "Hold" in rating:
        return f'<span style="color:orange; font-weight:bold;">{rating} ⏹️</span>'
    else:
        return f'<span style="color:red; font-weight:bold;">{rating} ❌</span>'

def html_progress_bar(value, max_value=100, bar_length=20, color='blue'):
    """Create an HTML progress bar"""
    percentage = min(100, max(0, (value / max_value * 100)))
    bar_filled = int(bar_length * percentage / 100)
    bar_empty = bar_length - bar_filled
    
    return f"""
    <div style="display:flex; align-items:center; margin-bottom:5px;">
        <div style="width:{bar_length * 10}px; height:15px; border:1px solid #ccc; border-radius:3px; overflow:hidden;">
            <div style="width:{percentage}%; height:100%; background-color:{color};"></div>
        </div>
        <span style="margin-left:10px;">{percentage:.1f}%</span>
    </div>
    """

def color_value(value, thresholds, colors):
    """Return value with color based on thresholds"""
    for i, threshold in enumerate(thresholds):
        if value <= threshold:
            return f'<span style="color:{colors[i]};">{value}</span>'
    return f'<span style="color:{colors[-1]};">{value}</span>'

In [7]:
# Create a list of dictionaries representing stocks with their attributes
stocks = [
    {
        "symbol": "AAPL",
        "current_price": 173.50,
        "ma_50": 168.30,
        "ma_200": 165.75,
        "pe_ratio": 28.5,
        "debt_equity": 1.2,
        "dividend_yield": 0.6,
        "sector": "Technology"
    },
    {
        "symbol": "MSFT",
        "current_price": 369.85,
        "ma_50": 358.20,
        "ma_200": 340.15,
        "pe_ratio": 33.8,
        "debt_equity": 0.4,
        "dividend_yield": 0.8,
        "sector": "Technology"
    },
    {
        "symbol": "JNJ",
        "current_price": 152.30,
        "ma_50": 156.40,
        "ma_200": 160.25,
        "pe_ratio": 15.2,
        "debt_equity": 0.5,
        "dividend_yield": 3.2,
        "sector": "Healthcare"
    },
    {
        "symbol": "JPM",
        "current_price": 178.20,
        "ma_50": 170.15,
        "ma_200": 155.40,
        "pe_ratio": 12.5,
        "debt_equity": 1.1,
        "dividend_yield": 2.4,
        "sector": "Financial"
    },
    {
        "symbol": "NFLX",
        "current_price": 604.75,
        "ma_50": 580.20,
        "ma_200": 520.35,
        "pe_ratio": 42.1,
        "debt_equity": 0.8,
        "dividend_yield": 0.0,
        "sector": "Technology"
    },
    {
        "symbol": "XOM",
        "current_price": 114.30,
        "ma_50": 118.60,
        "ma_200": 111.25,
        "pe_ratio": 9.2,
        "debt_equity": 0.2,
        "dividend_yield": 3.6,
        "sector": "Energy"
    },
    {
        "symbol": "WMT",
        "current_price": 59.80,
        "ma_50": 60.40,
        "ma_200": 55.75,
        "pe_ratio": 30.5,
        "debt_equity": 0.7,
        "dividend_yield": 1.5,
        "sector": "Retail"
    },
    {
        "symbol": "PFE",
        "current_price": 27.15,
        "ma_50": 29.30,
        "ma_200": 34.80,
        "pe_ratio": 7.4,
        "debt_equity": 0.6,
        "dividend_yield": 5.8,
        "sector": "Healthcare"
    }
]

In [8]:
# Main function to analyze stocks
def analyze_stocks(stock_list):
    """Analyze all stocks in the list based on screening criteria"""
    # Initialize counters for each rating category
    strong_buy_count = 0
    moderate_buy_count = 0
    hold_count = 0
    sell_count = 0
    
    # Store results to display later
    results = []
    
    # Iterate through each stock and apply screening logic
    for stock in stock_list:
        # Extract values for easier reference
        symbol = stock["symbol"]
        current_price = stock["current_price"]
        ma_50 = stock["ma_50"]
        ma_200 = stock["ma_200"]
        pe_ratio = stock["pe_ratio"]
        debt_equity = stock["debt_equity"]
        dividend_yield = stock["dividend_yield"]
        sector = stock["sector"]
        
        # Initialize scores for different aspects
        technical_score = 0
        value_score = 0
        financial_health_score = 0
        income_score = 0
        
        # Technical analysis
        if current_price > ma_50 and current_price > ma_200:
            technical_score += 2  # Price above both moving averages
        elif current_price > ma_50:
            technical_score += 1  # Price above 50-day MA but below 200-day MA
        elif current_price < ma_50 and current_price < ma_200:
            technical_score -= 2  # Price below both moving averages
        
        # Check for golden cross (50-day MA crossing above 200-day MA)
        if ma_50 > ma_200:
            technical_score += 1
        
        # Value analysis
        if pe_ratio < 15:
            value_score += 2  # Very low P/E ratio
        elif pe_ratio < 25:
            value_score += 1  # Moderate P/E ratio
        elif pe_ratio > 40:
            value_score -= 1  # High P/E ratio
        
        # Financial health analysis
        if debt_equity < 0.3:
            financial_health_score += 2  # Very low debt
        elif debt_equity < 0.7:
            financial_health_score += 1  # Moderate debt
        elif debt_equity > 1.5:
            financial_health_score -= 2  # High debt
        
        # Income analysis (dividends)
        if dividend_yield > 4.0:
            income_score += 2  # High dividend yield
        elif dividend_yield > 2.0:
            income_score += 1  # Moderate dividend yield
        
        # Calculate total score
        total_score = technical_score + value_score + financial_health_score + income_score
        
        # Determine rating based on total score
        if total_score >= 5:
            rating = "Strong Buy"
            strong_buy_count += 1
        elif total_score >= 2:
            rating = "Moderate Buy"
            moderate_buy_count += 1
        elif total_score >= -1:
            rating = "Hold"
            hold_count += 1
        else:
            rating = "Sell"
            sell_count += 1
        
        # Prepare technical analysis details
        tech_details = []
        if current_price > ma_50:
            tech_details.append(f"▲ Price (${current_price}) above 50-day MA (${ma_50})")
        else:
            tech_details.append(f"▼ Price (${current_price}) below 50-day MA (${ma_50})")
            
        if current_price > ma_200:
            tech_details.append(f"▲ Price (${current_price}) above 200-day MA (${ma_200})")
        else:
            tech_details.append(f"▼ Price (${current_price}) below 200-day MA (${ma_200})")
        
        if ma_50 > ma_200:
            tech_details.append(f"✓ Golden Cross: 50-day MA above 200-day MA")
        else:
            tech_details.append(f"✗ Death Cross: 50-day MA below 200-day MA")
        
        # Prepare explanation based on rating
        if rating == "Strong Buy":
            explanation = [
                "This stock shows strong technical and fundamental indicators.",
                "It has a positive trend and solid financial metrics."
            ]
        elif rating == "Moderate Buy":
            explanation = [
                "This stock has more positive indicators than negative ones.",
                "There are some cautions, but overall outlook is positive."
            ]
        elif rating == "Hold":
            explanation = [
                "This stock shows mixed signals in technical and fundamental analysis.",
                "There's no clear advantage to buying or selling at current levels."
            ]
        else:
            explanation = [
                "This stock has multiple negative indicators that suggest caution.",
                "Technical trends and/or fundamentals are unfavorable."
            ]
            
        # Store the results for this stock
        results.append({
            "symbol": symbol,
            "sector": sector,
            "current_price": current_price,
            "ma_50": ma_50, 
            "ma_200": ma_200,
            "rating": rating,
            "total_score": total_score,
            "technical_score": technical_score,
            "value_score": value_score,
            "financial_score": financial_health_score,
            "income_score": income_score,
            "tech_details": tech_details,
            "pe_ratio": pe_ratio,
            "debt_equity": debt_equity,
            "dividend_yield": dividend_yield,
            "explanation": explanation
        })
    
    # Sort results by total score (highest first)
    results.sort(key=lambda x: x["total_score"], reverse=True)
    
    return results, {
        "strong_buy": strong_buy_count,
        "moderate_buy": moderate_buy_count,
        "hold": hold_count,
        "sell": sell_count
    }

In [9]:
# Function to display individual stock analysis with charts
def display_stock_analysis(stock):
    """Display a detailed analysis for a single stock with visualizations"""
    symbol = stock["symbol"]
    sector = stock["sector"]
    current_price = stock["current_price"]
    rating = stock["rating"]
    tech_details = stock["tech_details"]
    
    # Create HTML for stock header
    header_html = f"""
    <div style="background-color:#f0f0f0; padding:15px; border-radius:10px; margin-bottom:15px; border-left:5px solid #004466;">
        <h3 style="margin:0; color:#004466;">📊 Analysis for {symbol} ({sector})</h3>
        <p style="margin:5px 0 0 0;">Current Price: <span style="font-weight:bold; color:navy;">${current_price}</span></p>
    </div>
    """
    display(HTML(header_html))
    
    # Create a figure with subplots for technical analysis
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
    
    # Create a price vs moving averages chart
    x = ['Current', '50-day MA', '200-day MA']
    y = [stock['current_price'], stock['ma_50'], stock['ma_200']]
    colors = ['blue', 'green', 'red']
    ax1.bar(x, y, color=colors, width=0.6, alpha=0.7)
    ax1.set_title('Price vs Moving Averages')
    ax1.set_ylabel('Price ($)')
    ax1.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Create score breakdown chart
    scores = [
        stock['technical_score'], 
        stock['value_score'], 
        stock['financial_score'],
        stock['income_score']
    ]
    score_labels = ['Technical', 'Value', 'Financial', 'Income']
    score_colors = ['green' if s > 0 else 'red' if s < 0 else 'gray' for s in scores]
    ax2.bar(score_labels, scores, color=score_colors, width=0.6, alpha=0.7)
    ax2.set_title('Score Components')
    ax2.set_ylabel('Score')
    ax2.grid(axis='y', linestyle='--', alpha=0.7)
    ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    
    # Add total score
    for i, score in enumerate(scores):
        if score != 0:
            ax2.text(i, score + (0.3 if score > 0 else -0.3), 
                     f"{score:+d}", ha='center', va='center', 
                     color='white', fontweight='bold',
                     bbox=dict(facecolor='black', alpha=0.5, boxstyle='round,pad=0.2'))
    
    plt.tight_layout()
    plt.show()
    
    # Display technical analysis details
    tech_html = "<div style='margin-top:15px;'><h4>Technical Analysis:</h4><ul>"
    for detail in tech_details:
        if "▲" in detail or "✓" in detail:
            tech_html += f"<li style='color:green;'>{detail}</li>"
        else:
            tech_html += f"<li style='color:red;'>{detail}</li>"
    tech_html += "</ul></div>"
    display(HTML(tech_html))
    
    # Display fundamental analysis details
    fundamental_html = """
    <div style='margin-top:15px;'>
        <h4>Fundamental Analysis:</h4>
        <table style="width:80%; border-collapse: collapse; margin-top:10px;">
            <tr style="background-color:#f2f2f2;">
                <th style="padding:8px; text-align:left; border:1px solid #ddd;">Metric</th>
                <th style="padding:8px; text-align:left; border:1px solid #ddd;">Value</th>
                <th style="padding:8px; text-align:left; border:1px solid #ddd;">Assessment</th>
            </tr>
    """
    
    # P/E Ratio
    pe_color = "green" if stock["pe_ratio"] < 15 else "orange" if stock["pe_ratio"] < 25 else "red"
    pe_assessment = "Attractive valuation" if stock["pe_ratio"] < 15 else "Reasonable valuation" if stock["pe_ratio"] < 25 else "Expensive valuation"
    
    # Debt/Equity
    debt_color = "green" if stock["debt_equity"] < 0.3 else "orange" if stock["debt_equity"] < 0.7 else "red"
    debt_assessment = "Very low debt" if stock["debt_equity"] < 0.3 else "Moderate debt" if stock["debt_equity"] < 0.7 else "High debt burden"
    
    # Dividend Yield
    div_color = "green" if stock["dividend_yield"] > 2.0 else "orange" if stock["dividend_yield"] > 0 else "gray"
    div_assessment = "High income" if stock["dividend_yield"] > 4.0 else "Moderate income" if stock["dividend_yield"] > 2.0 else "Low/No dividend income"
    
    fundamental_html += f"""
        <tr>
            <td style="padding:8px; border:1px solid #ddd;">P/E Ratio</td>
            <td style="padding:8px; border:1px solid #ddd; color:{pe_color}; font-weight:bold;">{stock["pe_ratio"]}</td>
            <td style="padding:8px; border:1px solid #ddd;">{pe_assessment}</td>
        </tr>
        <tr>
            <td style="padding:8px; border:1px solid #ddd;">Debt/Equity</td>
            <td style="padding:8px; border:1px solid #ddd; color:{debt_color}; font-weight:bold;">{stock["debt_equity"]}</td>
            <td style="padding:8px; border:1px solid #ddd;">{debt_assessment}</td>
        </tr>
        <tr>
            <td style="padding:8px; border:1px solid #ddd;">Dividend Yield</td>
            <td style="padding:8px; border:1px solid #ddd; color:{div_color}; font-weight:bold;">{stock["dividend_yield"]}%</td>
            <td style="padding:8px; border:1px solid #ddd;">{div_assessment}</td>
        </tr>
    """
    
    fundamental_html += "</table></div>"
    display(HTML(fundamental_html))
    
    # Display rating and explanation
    rating_color = {"Strong Buy": "green", "Moderate Buy": "blue", "Hold": "orange", "Sell": "red"}[rating]
    rating_symbol = {"Strong Buy": "🔥", "Moderate Buy": "✅", "Hold": "⏹️", "Sell": "❌"}[rating]
    
    rating_html = f"""
    <div style="margin-top:20px; padding:15px; border-radius:10px; background-color:{rating_color}; color:white;">
        <h3 style="margin:0;">{rating_symbol} RATING: {rating} {rating_symbol}</h3>
    </div>
    <div style="margin-top:10px; padding:15px; border-radius:10px; background-color:#f9f9f9;">
        <h4 style="margin-top:0;">Explanation:</h4>
        <ul>
    """
    
    for line in stock["explanation"]:
        rating_html += f"<li>{line}</li>"
    
    rating_html += """
        </ul>
    </div>
    """
    
    display(HTML(rating_html))
    
    # Display total score
    score_color = "green" if stock["total_score"] >= 2 else "red" if stock["total_score"] < 0 else "orange"
    score_html = f"""
    <div style="margin-top:15px; text-align:center;">
        <h3>Total Score: <span style="color:{score_color};">{stock["total_score"]:+d}</span></h3>
    </div>
    """
    
    display(HTML(score_html))

In [10]:
# Function to display summary results with visualizations
def display_summary(counts, results):
    """Display summary of screening results with charts"""
    display_section("SCREENING SUMMARY")
    
    # Calculate total
    total = sum(counts.values())
    
    if total > 0:
        # Create pie chart for category distribution
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Prepare data for pie chart
        categories = ["Strong Buy", "Moderate Buy", "Hold", "Sell"]
        values = [counts["strong_buy"], counts["moderate_buy"], counts["hold"], counts["sell"]]
        colors = ['green', 'blue', 'orange', 'red']
        explode = (0.1, 0, 0, 0)  # Explode the 1st slice (Strong Buy)
        
        # Plot pie chart
        ax1.pie(values, explode=explode, labels=categories, colors=colors, autopct='%1.1f%%',
                shadow=True, startangle=90)
        ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
        ax1.set_title('Distribution of Stock Ratings')
        
        # Create bar chart for average scores by category
        cat_tech_scores = []
        cat_value_scores = []
        cat_fin_scores = []
        cat_inc_scores = []
        
        # Calculate average scores for each category
        for cat in categories:
            cat_stocks = [s for s in results if s["rating"] == cat]
            
            if cat_stocks:
                cat_tech_scores.append(sum(s["technical_score"] for s in cat_stocks) / len(cat_stocks))
                cat_value_scores.append(sum(s["value_score"] for s in cat_stocks) / len(cat_stocks))
                cat_fin_scores.append(sum(s["financial_score"] for s in cat_stocks) / len(cat_stocks))
                cat_inc_scores.append(sum(s["income_score"] for s in cat_stocks) / len(cat_stocks))
            else:
                cat_tech_scores.append(0)
                cat_value_scores.append(0)
                cat_fin_scores.append(0)
                cat_inc_scores.append(0)
        
        # Create grouped bar chart
        x = range(len(categories))
        width = 0.2
        
        ax2.bar([i - width*1.5 for i in x], cat_tech_scores, width, label='Technical', color='purple', alpha=0.7)
        ax2.bar([i - width*0.5 for i in x], cat_value_scores, width, label='Value', color='blue', alpha=0.7)
        ax2.bar([i + width*0.5 for i in x], cat_fin_scores, width, label='Financial', color='green', alpha=0.7)
        ax2.bar([i + width*1.5 for i in x], cat_inc_scores, width, label='Income', color='orange', alpha=0.7)
        
        ax2.set_xticks(x)
        ax2.set_xticklabels(categories)
        ax2.set_ylabel('Average Score')
        ax2.set_title('Average Scores by Category')
        ax2.legend()
        ax2.grid(axis='y', linestyle='--', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Create HTML summary table
        summary_html = """
        <div style="margin-top:20px;">
            <table style="width:100%; border-collapse: collapse;">
                <tr style="background-color:#004466; color:white;">
                    <th style="padding:10px; text-align:left; border:1px solid #ddd;">Rating</th>
                    <th style="padding:10px; text-align:center; border:1px solid #ddd;">Count</th>
                    <th style="padding:10px; text-align:center; border:1px solid #ddd;">Percentage</th>
                    <th style="padding:10px; text-align:center; border:1px solid #ddd;">Distribution</th>
                </tr>
        """
        
        # Fill in table rows
        rating_data = [
            ("Strong Buy 🔥", counts["strong_buy"], "green"),
            ("Moderate Buy ✅", counts["moderate_buy"], "blue"),
            ("Hold ⏹️", counts["hold"], "orange"),
            ("Sell ❌", counts["sell"], "red")
        ]
        
        for rating, count, color in rating_data:
            percentage = (count / total) * 100 if total > 0 else 0
            
            summary_html += f"""
            <tr>
                <td style="padding:10px; border:1px solid #ddd; font-weight:bold; color:{color};">{rating}</td>
                <td style="padding:10px; border:1px solid #ddd; text-align:center;">{count}</td>
                <td style="padding:10px; border:1px solid #ddd; text-align:center;">{percentage:.1f}%</td>
                <td style="padding:10px; border:1px solid #ddd;">
                    <div style="width:100%; height:20px; background-color:#f0f0f0; border-radius:5px;">
                        <div style="width:{percentage}%; height:100%; background-color:{color}; border-radius:5px;"></div>
                    </div>
                </td>
            </tr>
            """
        
        summary_html += "</table></div>"
        display(HTML(summary_html))
        
        # Display top performers
        display_section("TOP PERFORMERS")
        
        # Convert to DataFrame for easier manipulation
        df = pd.DataFrame(results)
        
        # Create an interactive top performers table
        top_html = """
        <div style="margin-top:15px;">
            <table style="width:100%; border-collapse: collapse;">
                <tr style="background-color:#004466; color:white;">
                    <th style="padding:10px; text-align:left; border:1px solid #ddd;">Symbol</th>
                    <th style="padding:10px; text-align:center; border:1px solid #ddd;">Sector</th>
                    <th style="padding:10px; text-align:center; border:1px solid #ddd;">Price</th>
                    <th style="padding:10px; text-align:center; border:1px solid #ddd;">Rating</th>
                    <th style="padding:10px; text-align:center; border:1px solid #ddd;">Score</th>
                </tr>
        """
        
        # Add top 5 performers to table
        for _, row in df.head(5).iterrows():
            score_color = "green" if row["total_score"] >= 2 else "red" if row["total_score"] < 0 else "orange"
            top_html += f"""
            <tr>
                <td style="padding:10px; border:1px solid #ddd; font-weight:bold;">{row["symbol"]}</td>
                <td style="padding:10px; border:1px solid #ddd; text-align:center;">{row["sector"]}</td>
                <td style="padding:10px; border:1px solid #ddd; text-align:center;">${row["current_price"]}</td>
                <td style="padding:10px; border:1px solid #ddd; text-align:center;">{color_rating(row["rating"])}</td>
                <td style="padding:10px; border:1px solid #ddd; text-align:center; color:{score_color}; font-weight:bold;">{row["total_score"]:+d}</td>
            </tr>
            """
        
        top_html += "</table></div>"
        display(HTML(top_html))

In [11]:
# Function to display screening criteria
def display_criteria():
    """Display detailed information about the screening criteria"""
    display_section("SCREENING CRITERIA INFORMATION")
    
    criteria_html = """
    <div style="display:flex; flex-wrap:wrap; gap:20px; margin-top:15px;">
        <div style="flex:1; min-width:300px; background-color:#f8f8f8; border-radius:10px; padding:15px; border-left:5px solid #3366cc;">
            <h3 style="color:#3366cc; margin-top:0;">Technical Analysis</h3>
            <ul>
                <li>Price above both MAs: <span style="color:green; font-weight:bold;">+2 points</span></li>
                <li>Price above 50-day MA only: <span style="color:green; font-weight:bold;">+1 point</span></li>
                <li>Price below both MAs: <span style="color:red; font-weight:bold;">-2 points</span></li>
                <li>Golden Cross (50-MA > 200-MA): <span style="color:green; font-weight:bold;">+1 point</span></li>
            </ul>
        </div>
        
        <div style="flex:1; min-width:300px; background-color:#f8f8f8; border-radius:10px; padding:15px; border-left:5px solid #33cc99;">
            <h3 style="color:#33cc99; margin-top:0;">Value Analysis</h3>
            <ul>
                <li>P/E Ratio < 15: <span style="color:green; font-weight:bold;">+2 points</span> - Very attractive valuation</li>
                <li>P/E Ratio < 25: <span style="color:green; font-weight:bold;">+1 point</span> - Reasonable valuation</li>
                <li>P/E Ratio > 40: <span style="color:red; font-weight:bold;">-1 point</span> - Expensive valuation</li>
            </ul>
        </div>
        
        <div style="flex:1; min-width:300px; background-color:#f8f8f8; border-radius:10px; padding:15px; border-left:5px solid #cc6633;">
            <h3 style="color:#cc6633; margin-top:0;">Financial Health</h3>
            <ul>
                <li>Debt/Equity < 0.3: <span style="color:green; font-weight:bold;">+2 points</span> - Very low debt</li>
                <li>Debt/Equity < 0.7: <span style="color:green; font-weight:bold;">+1 point</span> - Moderate debt</li>
                <li>Debt/Equity > 1.5: <span style="color:red; font-weight:bold;">-2 points</span> - High debt burden</li>
            </ul>
        </div>
        
        <div style="flex:1; min-width:300px; background-color:#f8f8f8; border-radius:10px; padding:15px; border-left:5px solid #cc9933;">
            <h3 style="color:#cc9933; margin-top:0;">Income Potential</h3>
            <ul>
                <li>Dividend Yield > 4.0%: <span style="color:green; font-weight:bold;">+2 points</span> - High income</li>
                <li>Dividend Yield > 2.0%: <span style="color:green; font-weight:bold;">+1 point</span> - Moderate income</li>
                <li>Dividend Yield < 2.0%: <span style="color:gray;">0 points</span> - Low income</li>
            </ul>
        </div>
    </div>
    
    <div style="margin-top:30px; background-color:#f0f0f0; padding:20px; border-radius:10px;">
        <h3 style="margin-top:0;">Overall Rating Calculation</h3>
        <p>The final rating is determined by the sum of all individual scores:</p>
        <ul>
            <li><span style="color:green; font-weight:bold;">Strong Buy 🔥</span>: Total Score ≥ 5</li>
            <li><span style="color:blue; font-weight:bold;">Moderate Buy ✅</span>: Total Score ≥ 2</li>
            <li><span style="color:orange; font-weight:bold;">Hold ⏹️</span>: Total Score ≥ -1</li>
            <li><span style="color:red; font-weight:bold;">Sell ❌</span>: Total Score < -1</li>
        </ul>
    </div>
    
    <div style="margin-top:30px; background-color:#e6f7ff; padding:20px; border-radius:10px; border-left:5px solid #0099cc;">
        <h3 style="margin-top:0; color:#0099cc;">Interpretation Guide</h3>
        <ul>
            <li><strong>Strong Buy</strong>: Stock shows strong positive indicators across multiple criteria. Considered a prime candidate for purchase.</li>
            <li><strong>Moderate Buy</strong>: Stock has more positive than negative indicators. Worth considering for purchase but may have some cautions.</li>
            <li><strong>Hold</strong>: Stock shows mixed signals. Not a strong case for either buying or selling at current levels.</li>
            <li><strong>Sell</strong>: Stock has significant negative indicators. May be overvalued or facing fundamental/technical challenges.</li>
        </ul>
        <p style="margin-bottom:0;"><em>Note: This screening system is based on Chapter 4 concepts from <a href="https://www.amazon.com/dp/B0F3S8FQ7C" target="_blank">Practical Python for Effective Algorithmic Trading</a> and serves as an educational example.</em></p>
    </div>
    """
    
    display(HTML(criteria_html))

In [12]:
# Create an interactive menu for the stock screener
def run_interactive_screener():
    """Run the interactive stock screener with a menu system"""
    display_title()
    
    # Display interactive menu using HTML
    menu_html = """
    <div style="margin-top:20px; margin-bottom:20px;">
        <h2>Screening Options</h2>
        <div style="display:flex; flex-wrap:wrap; gap:15px;">
            <button 
                id="screen_all" 
                style="padding:10px 20px; background-color:#004466; color:white; border:none; border-radius:5px; cursor:pointer; font-size:16px;">
                Screen All Stocks
            </button>
            
            <button 
                id="screen_sector" 
                style="padding:10px 20px; background-color:#004466; color:white; border:none; border-radius:5px; cursor:pointer; font-size:16px;">
                Screen by Sector
            </button>
            
            <button 
                id="view_criteria" 
                style="padding:10px 20px; background-color:#004466; color:white; border:none; border-radius:5px; cursor:pointer; font-size:16px;">
                View Screening Criteria
            </button>
        </div>
    </div>
    """
    
    display(HTML(menu_html))
    
    # Since Jupyter doesn't support interactive buttons directly in output,
    # we'll provide alternative interactive functionality using Python input
    
    print("Select an option:")
    print("1: Screen All Stocks")
    print("2: Screen by Sector")
    print("3: View Screening Criteria")
    
    choice = input("Enter your choice (1-3): ")
    
    if choice == "1":
        # Screen all stocks
        display_section("SCREENING ALL STOCKS")
        results, counts = analyze_stocks(stocks)
        
        # Display each stock analysis
        for stock in results:
            display_stock_analysis(stock)
        
        # Display summary
        display_summary(counts, results)
        
    elif choice == "2":
        # Screen by sector
        display_section("SCREEN BY SECTOR")
        
        # Get unique sectors
        sectors = sorted(set(stock["sector"] for stock in stocks))
        
        print("Available sectors:")
        for i, sector in enumerate(sectors, 1):
            print(f"{i}. {sector}")
        
        # Get sector choice
        sector_choice = input("\nSelect a sector (number): ")
        try:
            selected_sector = sectors[int(sector_choice) - 1]
            
            # Filter stocks by sector
            sector_stocks = [stock for stock in stocks if stock["sector"] == selected_sector]
            
            display_section(f"SCREENING {selected_sector.upper()} SECTOR")
            
            # Analyze sector stocks
            results, counts = analyze_stocks(sector_stocks)
            
            # Display individual stock analysis
            for stock in results:
                display_stock_analysis(stock)
            
            # Display summary
            display_summary(counts, results)
            
        except (ValueError, IndexError):
            print("Invalid selection. Please try again.")
            
    elif choice == "3":
        # View screening criteria
        display_criteria()
        
    else:
        print("Invalid choice. Please restart the cell and try again.")

In [ ]:
# Run the interactive stock screener
run_interactive_screener()

## Extending Your Stock Screener

Now that you have a working stock screener, here are some ways you could extend it using concepts from future chapters:

1. **Data Integration (Chapter 7)**: 
   - Connect to a real-time stock data API to get current prices and metrics
   - Import stock data from CSV files or financial databases

2. **Advanced Visualization (Chapter 8)**:
   - Create more complex charts showing price trends with moving averages
   - Develop interactive visualizations that respond to user selection

3. **Technical Indicators (Chapter 9)**:
   - Add more sophisticated technical indicators like RSI, MACD, and Bollinger Bands
   - Integrate indicator crossover signals into your scoring system

4. **Complete Strategy (Chapter 10)**:
   - Develop a full trading strategy based on your screening criteria
   - Implement entry and exit rules with proper position sizing

5. **Backtesting (Chapter 11)**:
   - Test your screening criteria against historical data
   - Measure performance and refine your scoring system

For more comprehensive algorithmic trading techniques and a complete learning path, check out [Practical Python for Effective Algorithmic Trading](https://www.amazon.com/dp/B0F3S8FQ7C).

Have questions about implementing these enhancements? Join [The Quantitative Elite Community](https://www.skool.com/the-quantitative-elite) for support from fellow algorithmic traders.